In [1]:
import pandas as pd
item1 = pd.read_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/origin_df_1014.csv')

In [2]:
item1.columns[item1.columns.str.contains('pur_amount')]

Index(['pur_amount_1_1', 'pur_amount_1_2', 'pur_amount_2_1', 'pur_amount_2_2'], dtype='object')

In [3]:
item_1 = item1[['cust_no','가공식품_1', '가구/인테리어_1', '가전_1', '교육/문화용품_1', '기타_1', '명품_1', '신선식품_1',
       '외식_1', '의류_1', '의약품_1', '일상용품_1', '전문스포츠/레저_1', '패션잡화_1', '가공식품_2',
       '가구/인테리어_2', '가전_2', '교육/문화용품_2', '기타_2', '명품_2', '신선식품_2', '외식_2',
       '의류_2', '의약품_2', '일상용품_2', '전문스포츠/레저_2', '패션잡화_2']]

In [4]:
item_1.fillna(0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [5]:
# tqdm.pandas()
def qcut(series,cut=[0,0.04, 0.11, 0.23,0.4,0.6,0.77,0.89,0.96,1]):
    qcut = pd.qcut(series.loc[series != 0],cut).unique()
    return qcut


def rank(x, qcut,ascending = True):
    qcut = qcut.sort_values(ascending=ascending)
    if x == 0:
        return 0
        
    for idx,cut in enumerate(qcut):
        if x in cut:
            return idx +1 
        
# del_df['cut_amount_1'] = del_df['amount_1'].progress_apply(lambda x : rank(x, qcut(del_df['amount_1'])))
# del_df['cut_amount_2'] = del_df['amount_2'].progress_apply(lambda x : rank(x, qcut(del_df['amount_2'])))

def weight(df,col,new_name):
    origin_df = df.copy()
    qcut_1 = pd.qcut(origin_df.loc[origin_df[col]>0,col],[0,0.2,0.4,0.6,0.8,1]).unique()
    qcut_2 = pd.qcut(origin_df.loc[origin_df[col]<0,col],[0,0.2,0.4,0.6,0.8,1]).unique()

    origin_df.loc[origin_df[col]==0,new_name] = origin_df.loc[origin_df[col]==0,col].apply(lambda x : 0*rank(x, qcut_1))
    origin_df.loc[origin_df[col]>0,new_name] = origin_df.loc[origin_df[col]>0,col].apply(lambda x : 0.1*rank(x, qcut_1))
    origin_df.loc[origin_df[col]<0,new_name] = origin_df.loc[origin_df[col]<0,col].apply(lambda x : -0.1*rank(x, qcut_2,False))
    origin_df[new_name].fillna(0,inplace=True)
    return origin_df

In [6]:
for i in ['가공식품','의약품','신선식품',  '일상용품', '교육/문화용품', '외식', '패션잡화', '의류', '기타','전문스포츠/레저', '가구/인테리어', '가전', '명품']:

  qcut1 = qcut(item_1[f'{i}_1'])
  qcut2 = qcut(item_1[f'{i}_2'])

  item_1[f'cut_{i}_1'] = item_1[f'{i}_1'].apply(lambda x : rank(x, qcut1))
  item_1[f'cut_{i}_2'] = item_1[f'{i}_2'].apply(lambda x : rank(x, qcut2))

  item_1[f'item_{i}_ch'] = item_1[f'{i}_2'] - item_1[f'{i}_1']
  item_1 = weight(item_1,f'item_{i}_ch',f'item_ch_{i}_cut')

  item_1[f'item_{i}_weight']=item_1[[f'cut_{i}_1',f'cut_{i}_2',f'item_ch_{i}_cut']].apply(lambda x : (x[0]*x[2]) + (x[1]*x[2]/2),axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [7]:
item_1

,cust_no,가공식품_1,가구/인테리어_1,가전_1,교육/문화용품_1,기타_1,명품_1,신선식품_1,외식_1,의류_1,의약품_1,일상용품_1,전문스포츠/레저_1,패션잡화_1,가공식품_2,가구/인테리어_2,가전_2,교육/문화용품_2,기타_2,명품_2,신선식품_2,외식_2,의류_2,의약품_2,일상용품_2,전문스포츠/레저_2,패션잡화_2,cut_가공식품_1,cut_가공식품_2,item_가공식품_ch,item_ch_가공식품_cut,item_가공식품_weight,cut_의약품_1,cut_의약품_2,item_의약품_ch,item_ch_의약품_cut,item_의약품_weight,cut_신선식품_1,cut_신선식품_2,item_신선식품_ch,...,cut_외식_1,cut_외식_2,item_외식_ch,item_ch_외식_cut,item_외식_weight,cut_패션잡화_1,cut_패션잡화_2,item_패션잡화_ch,item_ch_패션잡화_cut,item_패션잡화_weight,cut_의류_1,cut_의류_2,item_의류_ch,item_ch_의류_cut,item_의류_weight,cut_기타_1,cut_기타_2,item_기타_ch,item_ch_기타_cut,item_기타_weight,cut_전문스포츠/레저_1,cut_전문스포츠/레저_2,item_전문스포츠/레저_ch,item_ch_전문스포츠/레저_cut,item_전문스포츠/레저_weight,cut_가구/인테리어_1,cut_가구/인테리어_2,item_가구/인테리어_ch,item_ch_가구/인테리어_cut,item_가구/인테리어_weight,cut_가전_1,cut_가전_2,item_가전_ch,item_ch_가전_cut,item_가전_weight,cut_명품_1,cut_명품_2,item_명품_ch,item_ch_명품_cut,item_명품_weight
0,1,3513830.0,177650.0,0.0,0.0,0.0,0.0,966425.0,288750.0,9612380.0,0.0,2209270.0,2143200.0,1013850.0,1112490.0,4770240.0,9062100.0,0.0,0.0,0.0,1265302.0,758800.0,11842680.0,8000.0,4707600.0,0.0,589100.0,9,6,-2401340.0,-0.5,-6.00,0,3,8000.0,0.2,0.30,6,7,298877.0,...,6,8,470050.0,0.5,5.00,7,6,-424750.0,-0.4,-4.0,8,8,2230300.0,0.5,6.00,0,0,0.0,0.0,0.0,8,0,-2143200.0,-0.5,-4.0,6,9,4592590.0,0.5,5.25,0,9,9062100.0,0.5,2.25,0,0,0.0,0.0,0.00
1,2,1527630.0,530700.0,1421700.0,0.0,0.0,0.0,3353800.0,472600.0,3577890.0,0.0,8692420.0,3949760.0,796730.0,1088150.0,39000.0,688000.0,0.0,0.0,434150.0,2184120.0,407500.0,6203330.0,0.0,7642020.0,2704920.0,530910.0,7,6,-439480.0,-0.5,-5.00,0,0,0.0,0.0,0.00,9,8,-1169680.0,...,7,7,-65100.0,-0.3,-3.15,7,6,-265820.0,-0.4,-4.0,7,7,2625440.0,0.5,5.25,0,0,0.0,0.0,0.0,9,8,-1244840.0,-0.5,-6.5,7,4,-491700.0,-0.5,-4.50,8,7,-733700.0,-0.5,-5.75,0,5,434150.0,0.3,0.75
2,3,616100.0,0.0,0.0,0.0,0.0,0.0,132165.0,0.0,80000.0,0.0,22950.0,0.0,0.0,652550.0,0.0,26900.0,0.0,0.0,0.0,253184.0,0.0,0.0,0.0,109920.0,0.0,0.0,4,4,36450.0,0.1,0.60,0,0,0.0,0.0,0.00,2,3,121019.0,...,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.0,3,0,-80000.0,-0.2,-0.60,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.00,0,4,26900.0,0.2,0.40,0,0,0.0,0.0,0.00
3,4,731750.0,5200.0,0.0,0.0,0.0,0.0,635526.0,246610.0,2305650.0,20320.0,174430.0,402100.0,520520.0,432260.0,421900.0,0.0,0.0,0.0,0.0,251990.0,177560.0,1448040.0,9800.0,808210.0,613350.0,94400.0,5,3,-299490.0,-0.4,-2.60,5,3,-10520.0,-0.3,-1.95,5,3,-383536.0,...,6,5,-69050.0,-0.4,-3.40,6,4,-426120.0,-0.4,-3.2,6,6,-857610.0,-0.4,-3.60,0,0,0.0,0.0,0.0,6,6,211250.0,0.3,2.7,1,6,416700.0,0.4,1.60,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.00
4,5,2016000.0,0.0,49900.0,0.0,0.0,57000.0,805820.0,474240.0,542900.0,0.0,1312640.0,63080.0,138570.0,917840.0,212300.0,0.0,0.0,0.0,0.0,268440.0,240100.0,166000.0,0.0,280850.0,71200.0,69000.0,8,5,-1098160.0,-0.5,-5.25,0,0,0.0,0.0,0.00,6,3,-537380.0,...,7,6,-234140.0,-0.5,-5.00,5,4,-69570.0,-0.2,-1.4,5,3,-376900.0,-0.3,-1.95,0,0,0.0,0.0,0.0,3,4,8120.0,0.1,0.5,0,6,212300.0,0.4,1.20,5,0,-49900.0,-0.3,-1.50,2,0,-57000.0,-0.1,-0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,19376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149420.0,0.0,0.0,0.0,0.0,0.0,122077.0,0.0,0.0,0.0,1800.0,0.0,0.0,0,1,149420.0,0.2,0.10,0,0,0.0,0.0,0.00,0,2,122077.0,...,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.00
19334,19377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16460.0,0.0,0.0,0.0,0.0,0.0,0.0,12500.0,15900.0,0.0,172500.0,0.0,0.0,0,1,16460.0,0.1,0.05,0,0,0.0,0.0,0.00,0,0,0.0,...,0,2,12500.0,0.1,0.10,0,0,0.0,0.0,0.0,0,1,15900.0,0.1,0.05,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.00,0,0,0.0,0.0,0.00
19335,19378,0.0,0.0,0.0,0.0,0.0,0.0

In [3]:
import pandas as pd
purprd = pd.read_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/purprd.csv')
purprd

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time,quarter,y_quarter,y_month
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420,가공식품,탄산음료,사이다,2014,2,토,1,주말,밤,2014_1,주말_밤,1,2014_1,2014_02
1,B,8664007,15,1504,B150401,11303,44,2014-02-22,13,2400,가공식품,탄산음료,사이다,2014,2,토,1,주말,오후,2014_1,주말_오후,1,2014_1,2014_02
2,B,8919520,15,1504,B150401,13210,48,2015-09-21,19,5250,가공식품,탄산음료,사이다,2015,9,월,2,주중,밤,2015_2,주중_밤,3,2015_3,2015_09
3,B,6956422,15,1504,B150401,13823,12,2014-08-13,18,2250,가공식품,탄산음료,사이다,2014,8,수,2,주중,오후,2014_2,주중_오후,3,2014_3,2014_08
4,B,6957156,15,1504,B150401,18970,12,2014-08-31,22,4900,가공식품,탄산음료,사이다,2014,8,일,2,주말,밤,2014_2,주말_밤,3,2014_3,2014_08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,B,7269975,14,1415,B141503,134,16,2014-11-30,15,8900,가공식품,전통과자,온라인팥빙수재료,2014,11,일,2,주말,오후,2014_2,주말_오후,4,2014_4,2014_11
28593026,B,7946814,68,6802,B680209,18484,26,2014-12-27,14,1000,패션잡화,여성양말,여성발가락,2014,12,토,2,주말,오후,2014_2,주말_오후,4,2014_4,2014_12
28593027,B,7777110,7,702,B070203,5043,21,2015-10-12,22,4510,가공식품,즉석구이안주,즉석쥐포,2015,10,월,2,주중,밤,2015_2,주중_밤,4,2015_4,2015_10
28593028,B,7579178,7,703,B070305,4073,18,2014-01-25,20,8900,가공식품,마른안주,온라인 안주,2014,1,토,1,주말,밤,2014_1,주말_밤,1,2014_1,2014_01


In [9]:
pr14_2 = purprd[purprd['period']=='2014_2']
pr15_1 = purprd[purprd['period']=='2015_1']

In [10]:
item2 = pd.pivot_table(pr14_2,index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').fillna(0).reset_index()
item3 = pd.pivot_table(pr15_1,index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').fillna(0).reset_index()

In [11]:
item_2 = pd.merge(item2, item3, how='outer',on='cust_no',suffixes=['_2','_3']).fillna(0)

In [12]:
for i in ['가공식품','의약품','신선식품',  '일상용품', '교육/문화용품', '외식', '패션잡화', '의류', '기타','전문스포츠/레저', '가구/인테리어', '가전', '명품']:

  qcut2 = qcut(item_2[f'{i}_2'])
  qcut3 = qcut(item_2[f'{i}_3'])

  item_2[f'cut_{i}_2'] = item_2[f'{i}_2'].apply(lambda x : rank(x, qcut2))
  item_2[f'cut_{i}_3'] = item_2[f'{i}_3'].apply(lambda x : rank(x, qcut3))

  item_2[f'item_{i}_ch'] = item_2[f'{i}_3'] - item_2[f'{i}_2']
  item_2 = weight(item_2,f'item_{i}_ch',f'item_ch_{i}_cut')

  item_2[f'item_{i}_weight']=item_2[[f'cut_{i}_2',f'cut_{i}_3',f'item_ch_{i}_cut']].apply(lambda x : (x[0]*x[2]/2) + (x[1]*x[2]),axis=1)

In [13]:
item_1.columns[item_1.columns.str.contains('weight')]

Index(['item_가공식품_weight', 'item_의약품_weight', 'item_신선식품_weight',
       'item_일상용품_weight', 'item_교육/문화용품_weight', 'item_외식_weight',
       'item_패션잡화_weight', 'item_의류_weight', 'item_기타_weight',
       'item_전문스포츠/레저_weight', 'item_가구/인테리어_weight', 'item_가전_weight',
       'item_명품_weight'],
      dtype='object')

In [14]:
item_2.columns[item_2.columns.str.contains('weight')]

Index(['item_가공식품_weight', 'item_의약품_weight', 'item_신선식품_weight',
       'item_일상용품_weight', 'item_교육/문화용품_weight', 'item_외식_weight',
       'item_패션잡화_weight', 'item_의류_weight', 'item_기타_weight',
       'item_전문스포츠/레저_weight', 'item_가구/인테리어_weight', 'item_가전_weight',
       'item_명품_weight'],
      dtype='object', name='cat_dae')

In [15]:
item = pd.merge(item_1[['cust_no','item_가공식품_weight', 'item_의약품_weight', 'item_신선식품_weight',
       'item_일상용품_weight', 'item_교육/문화용품_weight', 'item_외식_weight',
       'item_패션잡화_weight', 'item_의류_weight', 'item_기타_weight',
       'item_전문스포츠/레저_weight', 'item_가구/인테리어_weight', 'item_가전_weight',
       'item_명품_weight']],item_2[['cust_no','item_가공식품_weight', 'item_의약품_weight', 'item_신선식품_weight',
       'item_일상용품_weight', 'item_교육/문화용품_weight', 'item_외식_weight',
       'item_패션잡화_weight', 'item_의류_weight', 'item_기타_weight',
       'item_전문스포츠/레저_weight', 'item_가구/인테리어_weight', 'item_가전_weight',
       'item_명품_weight']],how='outer',on='cust_no',suffixes=['_1','_2']).fillna(0)
item

,cust_no,item_가공식품_weight_1,item_의약품_weight_1,item_신선식품_weight_1,item_일상용품_weight_1,item_교육/문화용품_weight_1,item_외식_weight_1,item_패션잡화_weight_1,item_의류_weight_1,item_기타_weight_1,item_전문스포츠/레저_weight_1,item_가구/인테리어_weight_1,item_가전_weight_1,item_명품_weight_1,item_가공식품_weight_2,item_의약품_weight_2,item_신선식품_weight_2,item_일상용품_weight_2,item_교육/문화용품_weight_2,item_외식_weight_2,item_패션잡화_weight_2,item_의류_weight_2,item_기타_weight_2,item_전문스포츠/레저_weight_2,item_가구/인테리어_weight_2,item_가전_weight_2,item_명품_weight_2
0,1,-6.00,0.30,3.80,6.25,0.0,5.00,-4.0,6.00,0.0,-4.0,5.25,2.25,0.00,0.90,1.95,-3.80,-5.25,0.0,-3.60,5.0,-4.80,0.0,3.0,-5.75,-5.25,1.50
1,2,-5.00,0.00,-6.50,-6.75,0.0,-3.15,-4.0,5.25,0.0,-6.5,-4.50,-5.75,0.75,-2.40,3.00,-6.00,6.75,0.0,-4.25,-1.8,5.75,0.0,-4.8,1.40,5.75,2.55
2,3,0.60,0.00,0.70,0.40,0.0,0.00,0.0,-0.60,0.0,0.0,0.00,0.40,0.00,-2.00,0.00,-0.70,-0.40,0.0,0.00,0.0,0.00,0.0,0.0,0.00,-0.40,0.00
3,4,-2.60,-1.95,-3.25,3.00,0.0,-3.40,-3.2,-3.60,0.0,2.7,1.60,0.00,0.00,1.65,-0.45,3.25,-2.80,0.0,4.75,-1.0,-0.90,0.0,-2.8,-1.20,0.00,0.00
4,5,-5.25,0.00,-3.75,-4.50,0.0,-5.00,-1.4,-1.95,0.0,0.5,1.20,-1.50,-0.20,4.75,0.20,-0.45,2.10,0.0,-1.80,-0.4,1.95,0.0,-0.4,-1.20,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19367,19283,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,3.50,0.00,0.10,4.50,0.0,1.60,2.4,0.30,0.0,0.0,0.00,0.00,0.00
19368,19350,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.80,2.00,0.00,3.50,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00
19369,19381,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,2.50,0.60,3.50,0.90,0.1,0.60,0.8,1.20,0.0,0.3,1.50,0.00,0.00
19370,19382,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,2.00,3.50,2.50,0.40,0.6,2.00,0.3,0.30,0.0,1.2,0.10,0.00,0.00


In [16]:
purprd.cat_dae.unique()

array(['가공식품', '의약품', '신선식품', '일상용품', '교육/문화용품', '외식', '패션잡화', '의류', '기타',
       '전문스포츠/레저', '가구/인테리어', '가전', '명품'], dtype=object)

In [17]:
for i in ['가공식품', '의약품', '신선식품', '일상용품', '교육/문화용품', '외식', '패션잡화', '의류', '기타','전문스포츠/레저', '가구/인테리어', '가전', '명품']:
  item[f'{i}'] = item[f'item_{i}_weight_1'] + item[f'item_{i}_weight_2']

In [18]:
item.columns[item.columns.str.contains('[가-힣]')]

Index(['item_가공식품_weight_1', 'item_의약품_weight_1', 'item_신선식품_weight_1',
       'item_일상용품_weight_1', 'item_교육/문화용품_weight_1', 'item_외식_weight_1',
       'item_패션잡화_weight_1', 'item_의류_weight_1', 'item_기타_weight_1',
       'item_전문스포츠/레저_weight_1', 'item_가구/인테리어_weight_1', 'item_가전_weight_1',
       'item_명품_weight_1', 'item_가공식품_weight_2', 'item_의약품_weight_2',
       'item_신선식품_weight_2', 'item_일상용품_weight_2', 'item_교육/문화용품_weight_2',
       'item_외식_weight_2', 'item_패션잡화_weight_2', 'item_의류_weight_2',
       'item_기타_weight_2', 'item_전문스포츠/레저_weight_2', 'item_가구/인테리어_weight_2',
       'item_가전_weight_2', 'item_명품_weight_2', '가공식품', '의약품', '신선식품', '일상용품',
       '교육/문화용품', '외식', '패션잡화', '의류', '기타', '전문스포츠/레저', '가구/인테리어', '가전', '명품'],
      dtype='object')

In [19]:
test_item = item[['cust_no','가공식품', '의약품', '신선식품', '일상용품','교육/문화용품', '외식', '패션잡화', '의류', '기타', '전문스포츠/레저', '가구/인테리어', '가전', '명품']]

In [20]:
test_item

,cust_no,가공식품,의약품,신선식품,일상용품,교육/문화용품,외식,패션잡화,의류,기타,전문스포츠/레저,가구/인테리어,가전,명품
0,1,-5.10,2.25,0.0,1.0,0.0,1.40,1.0,1.2,0.0,-1.0,-0.5,-3.0,1.5
1,2,-7.40,3.00,-12.5,0.0,0.0,-7.40,-5.8,11.0,0.0,-11.3,-3.1,0.0,3.3
2,3,-1.40,0.00,0.0,0.0,0.0,0.00,0.0,-0.6,0.0,0.0,0.0,0.0,0.0
3,4,-0.95,-2.40,0.0,0.2,0.0,1.35,-4.2,-4.5,0.0,-0.1,0.4,0.0,0.0
4,5,-0.50,0.20,-4.2,-2.4,0.0,-6.80,-1.8,0.0,0.0,0.1,0.0,-1.5,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19367,19283,3.50,0.00,0.1,4.5,0.0,1.60,2.4,0.3,0.0,0.0,0.0,0.0,0.0
19368,19350,0.80,2.00,0.0,3.5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19369,19381,2.50,0.60,3.5,0.9,0.1,0.60,0.8,1.2,0.0,0.3,1.5,0.0,0.0
19370,19382,2.00,3.50,2.5,0.4,0.6,2.00,0.3,0.3,0.0,1.2,0.1,0.0,0.0


In [22]:
pur_1 = item1[['cust_no','pur_amount_1_1', 'pur_amount_1_2', 'pur_amount_2_1', 'pur_amount_2_2']]

In [23]:
pur_1

,cust_no,pur_amount_1_1,pur_amount_1_2,pur_amount_2_1,pur_amount_2_2
0,1,9141590.0,10783765.0,10781550.0,23334762.0
1,2,14123470.0,10199760.0,14841180.0,7080920.0
2,3,302782.0,548433.0,648188.0,394366.0
3,4,1309643.0,3732463.0,2434410.0,1823100.0
4,5,3145330.0,2314820.0,1827290.0,398440.0
...,...,...,...,...,...
19333,19376,0.0,0.0,0.0,273297.0
19334,19377,0.0,0.0,0.0,217360.0
19335,19378,0.0,0.0,0.0,474760.0
19336,19379,0.0,0.0,130300.0,1060320.0


In [24]:
for i in [1,2]:

  qcut1 = qcut(pur_1[f'pur_amount_{i}_1'])
  qcut2 = qcut(pur_1[f'pur_amount_{i}_2'])

  pur_1[f'cut_{i}_1'] = pur_1[f'pur_amount_{i}_1'].apply(lambda x : rank(x, qcut1))
  pur_1[f'cut_{i}_2'] = pur_1[f'pur_amount_{i}_1'].apply(lambda x : rank(x, qcut2))

  pur_1[f'pur_amount_{i}_ch'] = pur_1[f'pur_amount_{i}_2'] - pur_1[f'pur_amount_{i}_1']
  pur_1 = weight(pur_1,f'pur_amount_{i}_ch',f'pur_amount_{i}_cut')

  pur_1[f'pur_amount_{i}_weight']=pur_1[[f'cut_{i}_1',f'cut_{i}_2',f'pur_amount_{i}_cut']].apply(lambda x : (x[0]*x[2]) + (x[1]*x[2]/2),axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [25]:
pur_1.columns[pur_1.columns.str.contains('weight')]

Index(['pur_amount_1_weight', 'pur_amount_2_weight'], dtype='object')

In [26]:
pr_2_1 = purprd[purprd['y_quarter']=='2014_3']
pr_2_2 = purprd[purprd['y_quarter']=='2014_4']
pr_3_1 = purprd[purprd['y_quarter']=='2015_1']
pr_3_2 = purprd[purprd['y_quarter']=='2015_2']

In [27]:
quarter_1 = pd.pivot_table(pr_2_1,index='cust_no',values='pur_amount',aggfunc='sum').reset_index()
quarter_2 = pd.pivot_table(pr_2_2,index='cust_no',values='pur_amount',aggfunc='sum').reset_index()
quarter_3 = pd.pivot_table(pr_3_1,index='cust_no',values='pur_amount',aggfunc='sum').reset_index()
quarter_4 = pd.pivot_table(pr_3_2,index='cust_no',values='pur_amount',aggfunc='sum').reset_index()

In [28]:
period_1 = pd.merge(quarter_1, quarter_2, how='outer',on='cust_no',suffixes=['_2_1','_2_2'])
period_2 = pd.merge(quarter_3, quarter_4, how='outer',on='cust_no',suffixes=['_3_1','_3_2'])

In [29]:
period = pd.merge(period_1, period_2, how='outer',on='cust_no').fillna(0)
period

,cust_no,pur_amount_2_1,pur_amount_2_2,pur_amount_3_1,pur_amount_3_2
0,1,10781550.0,23334762.0,9302985.0,8595380.0
1,2,14841180.0,7080920.0,18481480.0,9310130.0
2,3,648188.0,394366.0,279310.0,312970.0
3,4,2434410.0,1823100.0,1737292.0,1919645.0
4,5,1827290.0,398440.0,2124430.0,810500.0
...,...,...,...,...,...
19351,19228,0.0,0.0,0.0,1964420.0
19352,19232,0.0,0.0,0.0,24770.0
19353,19244,0.0,0.0,0.0,190810.0
19354,19283,0.0,0.0,0.0,6228150.0


In [30]:
for i in [2,3]:

  qcut1 = qcut(period[f'pur_amount_{i}_1'])
  qcut2 = qcut(period[f'pur_amount_{i}_2'])

  period[f'cut_{i}_1'] = period[f'pur_amount_{i}_1'].apply(lambda x : rank(x, qcut1))
  period[f'cut_{i}_2'] = period[f'pur_amount_{i}_2'].apply(lambda x : rank(x, qcut2))

  period[f'pur_amount_period_{i}_ch'] = period[f'pur_amount_{i}_2'] - period[f'pur_amount_{i}_1']
  period = weight(period,f'pur_amount_period_{i}_ch',f'pur_amount_period_ch_{i}_cut')

  period[f'pur_amount_period_{i}_weight']=period[[f'cut_{i}_2',f'cut_{i}_1',f'pur_amount_period_ch_{i}_cut']].apply(lambda x : (x[0]*x[2]/2) + (x[1]*x[2]),axis=1)

In [31]:
pur1 = pur_1[['cust_no','pur_amount_1_weight', 'pur_amount_2_weight']]

In [32]:
pur2 = period[['cust_no','pur_amount_period_2_weight', 'pur_amount_period_3_weight']]

In [33]:
period.columns[period.columns.str.contains('weight')]

Index(['pur_amount_period_2_weight', 'pur_amount_period_3_weight'], dtype='object')

In [34]:
pur_period = pd.merge(pur1,pur2,how='outer',on='cust_no').fillna(0)
pur_period

,cust_no,pur_amount_1_weight,pur_amount_2_weight,pur_amount_period_2_weight,pur_amount_period_3_weight
0,1,4.8,5.75,6.25,-3.45
1,2,-6.0,-6.50,-6.25,-6.50
2,3,0.3,-0.60,-0.50,0.15
3,4,3.0,-2.25,-2.25,1.50
4,5,-3.6,-3.00,-2.20,-2.40
...,...,...,...,...,...
19367,19138,0.0,0.00,0.00,0.05
19368,19208,0.0,0.00,0.00,1.50
19369,19244,0.0,0.00,0.00,0.10
19370,19283,0.0,0.00,0.00,1.75


In [35]:
pur_period.columns

Index(['cust_no', 'pur_amount_1_weight', 'pur_amount_2_weight',
       'pur_amount_period_2_weight', 'pur_amount_period_3_weight'],
      dtype='object')

In [37]:
purprd[(purprd['year']=='2014')or(purprd['quarter']=='2015_1')]

ValueError: ignored

In [39]:
purprd

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time,quarter,y_quarter,y_month
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420,가공식품,탄산음료,사이다,2014,2,토,1,주말,밤,2014_1,주말_밤,1,2014_1,2014_02
1,B,8664007,15,1504,B150401,11303,44,2014-02-22,13,2400,가공식품,탄산음료,사이다,2014,2,토,1,주말,오후,2014_1,주말_오후,1,2014_1,2014_02
2,B,8919520,15,1504,B150401,13210,48,2015-09-21,19,5250,가공식품,탄산음료,사이다,2015,9,월,2,주중,밤,2015_2,주중_밤,3,2015_3,2015_09
3,B,6956422,15,1504,B150401,13823,12,2014-08-13,18,2250,가공식품,탄산음료,사이다,2014,8,수,2,주중,오후,2014_2,주중_오후,3,2014_3,2014_08
4,B,6957156,15,1504,B150401,18970,12,2014-08-31,22,4900,가공식품,탄산음료,사이다,2014,8,일,2,주말,밤,2014_2,주말_밤,3,2014_3,2014_08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593025,B,7269975,14,1415,B141503,134,16,2014-11-30,15,8900,가공식품,전통과자,온라인팥빙수재료,2014,11,일,2,주말,오후,2014_2,주말_오후,4,2014_4,2014_11
28593026,B,7946814,68,6802,B680209,18484,26,2014-12-27,14,1000,패션잡화,여성양말,여성발가락,2014,12,토,2,주말,오후,2014_2,주말_오후,4,2014_4,2014_12
28593027,B,7777110,7,702,B070203,5043,21,2015-10-12,22,4510,가공식품,즉석구이안주,즉석쥐포,2015,10,월,2,주중,밤,2015_2,주중_밤,4,2015_4,2015_10
28593028,B,7579178,7,703,B070305,4073,18,2014-01-25,20,8900,가공식품,마른안주,온라인 안주,2014,1,토,1,주말,밤,2014_1,주말_밤,1,2014_1,2014_01


In [41]:
pr14_1 = purprd[purprd['period']=='2014_1']
pr14_1

,channel,receipt_no,code_dae,code_jung,code_so,cust_no,branch_no,pur_date,pur_hour,pur_amount,cat_dae,cat_jung,cat_so,year,month,yoil,half,week,pur_time,period,pur_week_time,quarter,y_quarter,y_month
0,B,8664000,15,1504,B150401,17218,44,2014-02-22,20,2420,가공식품,탄산음료,사이다,2014,2,토,1,주말,밤,2014_1,주말_밤,1,2014_1,2014_02
1,B,8664007,15,1504,B150401,11303,44,2014-02-22,13,2400,가공식품,탄산음료,사이다,2014,2,토,1,주말,오후,2014_1,주말_오후,1,2014_1,2014_02
21,B,9427551,15,1504,B150401,14795,71,2014-04-08,20,6400,가공식품,탄산음료,사이다,2014,4,화,1,주중,밤,2014_1,주중_밤,2,2014_2,2014_04
26,B,8198575,15,1504,B150401,1310,32,2014-01-25,21,4050,가공식품,탄산음료,사이다,2014,1,토,1,주말,밤,2014_1,주말_밤,1,2014_1,2014_01
27,B,8345963,15,1504,B150401,10050,34,2014-06-21,22,2650,가공식품,탄산음료,사이다,2014,6,토,1,주말,밤,2014_1,주말_밤,2,2014_2,2014_06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28593004,B,2059,11,1105,B110506,6798,1,2014-01-28,14,24590,가공식품,조미선물세트,기타세트,2014,1,화,1,주중,오후,2014_1,주중_오후,1,2014_1,2014_01
28593017,B,70929,45,4518,B451803,3229,3,2014-05-10,18,35000,패션잡화,썬글라스,온라인 썬글라스,2014,5,토,1,주말,오후,2014_1,주말_오후,2,2014_2,2014_05
28593023,B,9922930,5,506,B050607,9039,90,2014-01-02,20,11800,가공식품,약초,둥글레,2014,1,목,1,주중,밤,2014_1,주중_밤,1,2014_1,2014_01
28593024,B,10496610,39,3901,B390103,11219,113,2014-03-03,15,860000,가전,냉장고,기능성냉장고,2014,3,월,1,주중,오후,2014_1,주중_오후,1,2014_1,2014_03


In [47]:
A_1 = pd.pivot_table(pr14_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['A'].reset_index().fillna(0)
B_1 = pd.pivot_table(pr14_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['B'].reset_index().fillna(0)
C_1 = pd.pivot_table(pr14_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['C'].reset_index().fillna(0)

In [48]:
A_2 = pd.pivot_table(pr14_2,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['A'].reset_index().fillna(0)
B_2 = pd.pivot_table(pr14_2,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['B'].reset_index().fillna(0)
C_2 = pd.pivot_table(pr14_2,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['C'].reset_index().fillna(0)

In [49]:
A_3 = pd.pivot_table(pr15_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['A'].reset_index().fillna(0)
B_3 = pd.pivot_table(pr15_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['B'].reset_index().fillna(0)
C_3 = pd.pivot_table(pr15_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum')['C'].reset_index().fillna(0)

In [55]:
A_train = pd.merge(A_1,A_2,how='outer',on='cust_no',suffixes=['_A_1','_A_2']).fillna(0)
B_train = pd.merge(B_1,B_2,how='outer',on='cust_no',suffixes=['_B_1','_B_2']).fillna(0)
C_train = pd.merge(C_1,C_2,how='outer',on='cust_no',suffixes=['_C_1','_C_2']).fillna(0)

In [63]:
A_train1 = pd.merge(A_1,A_2,how='outer',on='cust_no',suffixes=['_A_1','_A_2']).fillna(0)
B_train1 = pd.merge(B_1,B_2,how='outer',on='cust_no',suffixes=['_B_1','_B_2']).fillna(0)
C_train1 = pd.merge(C_1,C_2,how='outer',on='cust_no',suffixes=['_C_1','_C_2']).fillna(0)

In [56]:
A_test = pd.merge(A_2,A_3,how='outer',on='cust_no',suffixes=['_A_2','_A_3']).fillna(0)
B_test = pd.merge(B_2,B_3,how='outer',on='cust_no',suffixes=['_B_2','_B_3']).fillna(0)
C_test = pd.merge(C_2,C_3,how='outer',on='cust_no',suffixes=['_C_2','_C_3']).fillna(0)

In [57]:
A_train

cat_dae,cust_no,가공식품_A_1,가구/인테리어_A_1,가전_A_1,교육/문화용품_A_1,기타_A_1,명품_A_1,신선식품_A_1,외식_A_1,의류_A_1,일상용품_A_1,전문스포츠/레저_A_1,패션잡화_A_1,가공식품_A_2,가구/인테리어_A_2,가전_A_2,교육/문화용품_A_2,기타_A_2,명품_A_2,신선식품_A_2,외식_A_2,의류_A_2,일상용품_A_2,전문스포츠/레저_A_2,패션잡화_A_2
0,1,3492840.0,177650.0,0.0,0.0,0.0,0.0,787820.0,266950.0,9612380.0,2208520.0,2143200.0,1013850.0,914680.0,4770240.0,9062100.0,0.0,0.0,0.0,924860.0,702400.0,11842680.0,4534240.0,0.0,589100.0
1,2,1527630.0,530700.0,1421700.0,0.0,0.0,0.0,3353800.0,472600.0,3577890.0,8661920.0,3949760.0,796730.0,1088150.0,39000.0,688000.0,0.0,0.0,434150.0,2184120.0,407500.0,6203330.0,7595570.0,2704920.0,530910.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,659890.0,0.0,0.0,0.0,0.0,0.0,367480.0,246610.0,2272650.0,159010.0,402100.0,520520.0,301290.0,404000.0,0.0,0.0,0.0,0.0,91530.0,177560.0,1323740.0,775050.0,613350.0,25000.0
4,5,2016000.0,0.0,49900.0,0.0,0.0,57000.0,789820.0,474240.0,542900.0,1312640.0,63080.0,138570.0,917840.0,212300.0,0.0,0.0,0.0,0.0,268440.0,240100.0,166000.0,280850.0,71200.0,69000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,19376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19334,19377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,12500.0,15900.0,0.0,0.0,0.0
19335,19378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23800.0,0.0,0.0,0.0,0.0,0.0,67000.0,8500.0,151240.0,163220.0,0.0,0.0
19336,19379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,449900.0


In [60]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['A']:
    qcut2 = qcut(A_train[f'{i}_{j}_1'])
    qcut3 = qcut(A_train[f'{i}_{j}_2'])

    A_train[f'cut_{i}_{j}_1'] = A_train[f'{i}_{j}_1'].apply(lambda x : rank(x, qcut2))
    A_train[f'cut_{i}_{j}_2'] = A_train[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut3))

    A_train[f'item_{i}_{j}_ch'] = A_train[f'{i}_{j}_2'] - A_train[f'{i}_{j}_1']
    A_train = weight(A_train,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    A_train[f'item_{i}_{j}_weight']=A_train[[f'cut_{i}_{j}_1',f'cut_{i}_{j}_2',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0] + x[1])*x[2],axis=1)

In [61]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['B']:
    qcut2 = qcut(B_train[f'{i}_{j}_1'])
    qcut3 = qcut(B_train[f'{i}_{j}_2'])

    B_train[f'cut_{i}_{j}_1'] = B_train[f'{i}_{j}_1'].apply(lambda x : rank(x, qcut2))
    B_train[f'cut_{i}_{j}_2'] = B_train[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut3))

    B_train[f'item_{i}_{j}_ch'] = B_train[f'{i}_{j}_2'] - B_train[f'{i}_{j}_1']
    B_train = weight(B_train,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    B_train[f'item_{i}_{j}_weight']=B_train[[f'cut_{i}_{j}_1',f'cut_{i}_{j}_2',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0] + x[1])*x[2],axis=1)

In [62]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['C']:
    qcut2 = qcut(C_train[f'{i}_{j}_1'])
    qcut3 = qcut(C_train[f'{i}_{j}_2'])

    C_train[f'cut_{i}_{j}_1'] = C_train[f'{i}_{j}_1'].apply(lambda x : rank(x, qcut2))
    C_train[f'cut_{i}_{j}_2'] = C_train[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut3))

    C_train[f'item_{i}_{j}_ch'] = C_train[f'{i}_{j}_2'] - C_train[f'{i}_{j}_1']
    C_train = weight(C_train,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    C_train[f'item_{i}_{j}_weight']=C_train[[f'cut_{i}_{j}_1',f'cut_{i}_{j}_2',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0] + x[1])*x[2],axis=1)

In [65]:
A_train.columns[A_train.columns.str.contains('weight')]

Index(['item_가공식품_A_weight', 'item_신선식품_A_weight', 'item_일상용품_A_weight',
       'item_의류_A_weight'],
      dtype='object', name='cat_dae')

In [69]:
train_item_weight = pd.merge(A_train[['cust_no','item_가공식품_A_weight', 'item_신선식품_A_weight', 'item_일상용품_A_weight',
       'item_의류_A_weight']],B_train[['cust_no','item_가공식품_B_weight', 'item_신선식품_B_weight', 'item_일상용품_B_weight',
       'item_의류_B_weight']],how='outer',on='cust_no').fillna(0)

In [72]:
train_item_weight = pd.merge(train_item_weight,C_train[['cust_no','item_가공식품_C_weight', 'item_신선식품_C_weight', 'item_일상용품_C_weight',
       'item_의류_C_weight']],how='outer',on='cust_no').fillna(0)

In [73]:
train_item_weight.to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/train_item_weight.csv')

In [75]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['A']:
    qcut2 = qcut(A_train1[f'{i}_{j}_1'])
    qcut3 = qcut(A_train1[f'{i}_{j}_2'])

    A_train1[f'cut_{i}_{j}_1'] = A_train1[f'{i}_{j}_1'].apply(lambda x : rank(x, qcut2))
    A_train1[f'cut_{i}_{j}_2'] = A_train1[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut3))

    A_train1[f'item_{i}_{j}_ch'] = A_train1[f'{i}_{j}_2'] - A_train1[f'{i}_{j}_1']
    A_train1 = weight(A_train1,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    A_train1[f'item_{i}_{j}_weight']=A_train1[[f'cut_{i}_{j}_1',f'cut_{i}_{j}_2',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0]*x[2]) + (x[1]*x[2]/2),axis=1)

In [76]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['B']:
    qcut2 = qcut(B_train1[f'{i}_{j}_1'])
    qcut3 = qcut(B_train1[f'{i}_{j}_2'])

    B_train1[f'cut_{i}_{j}_1'] = B_train1[f'{i}_{j}_1'].apply(lambda x : rank(x, qcut2))
    B_train1[f'cut_{i}_{j}_2'] = B_train1[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut3))

    B_train1[f'item_{i}_{j}_ch'] = B_train1[f'{i}_{j}_2'] - B_train1[f'{i}_{j}_1']
    B_train1 = weight(B_train1,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    B_train1[f'item_{i}_{j}_weight']=B_train1[[f'cut_{i}_{j}_1',f'cut_{i}_{j}_2',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0]*x[2]) + (x[1]*x[2]/2),axis=1)

In [77]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['C']:
    qcut2 = qcut(C_train1[f'{i}_{j}_1'])
    qcut3 = qcut(C_train1[f'{i}_{j}_2'])

    C_train1[f'cut_{i}_{j}_1'] = C_train1[f'{i}_{j}_1'].apply(lambda x : rank(x, qcut2))
    C_train1[f'cut_{i}_{j}_2'] = C_train1[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut3))

    C_train1[f'item_{i}_{j}_ch'] = C_train1[f'{i}_{j}_2'] - C_train1[f'{i}_{j}_1']
    C_train1 = weight(C_train1,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    C_train1[f'item_{i}_{j}_weight']=C_train1[[f'cut_{i}_{j}_1',f'cut_{i}_{j}_2',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0]*x[2]) + (x[1]*x[2]/2),axis=1)

In [79]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['A']:
    qcut2 = qcut(A_test[f'{i}_{j}_2'])
    qcut3 = qcut(A_test[f'{i}_{j}_3'])

    A_test[f'cut_{i}_{j}_2'] = A_test[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut2))
    A_test[f'cut_{i}_{j}_3'] = A_test[f'{i}_{j}_3'].apply(lambda x : rank(x, qcut3))

    A_test[f'item_{i}_{j}_ch'] = A_test[f'{i}_{j}_3'] - A_test[f'{i}_{j}_2']
    A_test = weight(A_test,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    A_test[f'item_{i}_{j}_weight']=A_test[[f'cut_{i}_{j}_2',f'cut_{i}_{j}_3',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0]*x[2]/2) + (x[1]*x[2]),axis=1)

In [80]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['B']:
    qcut2 = qcut(B_test[f'{i}_{j}_2'])
    qcut3 = qcut(B_test[f'{i}_{j}_3'])

    B_test[f'cut_{i}_{j}_2'] = B_test[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut2))
    B_test[f'cut_{i}_{j}_3'] = B_test[f'{i}_{j}_3'].apply(lambda x : rank(x, qcut3))

    B_test[f'item_{i}_{j}_ch'] = B_test[f'{i}_{j}_3'] - B_test[f'{i}_{j}_2']
    B_test = weight(B_test,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    B_test[f'item_{i}_{j}_weight']=B_test[[f'cut_{i}_{j}_2',f'cut_{i}_{j}_3',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0]*x[2]/2) + (x[1]*x[2]),axis=1)

In [81]:
for i in ['가공식품','신선식품',  '일상용품', '의류']:
  for j in ['C']:
    qcut2 = qcut(C_test[f'{i}_{j}_2'])
    qcut3 = qcut(C_test[f'{i}_{j}_3'])

    C_test[f'cut_{i}_{j}_2'] = C_test[f'{i}_{j}_2'].apply(lambda x : rank(x, qcut2))
    C_test[f'cut_{i}_{j}_3'] = C_test[f'{i}_{j}_3'].apply(lambda x : rank(x, qcut3))

    C_test[f'item_{i}_{j}_ch'] = C_test[f'{i}_{j}_3'] - C_test[f'{i}_{j}_2']
    C_test = weight(C_test,f'item_{i}_{j}_ch',f'item_ch_{i}_{j}_cut')

    C_test[f'item_{i}_{j}_weight']=C_test[[f'cut_{i}_{j}_2',f'cut_{i}_{j}_3',f'item_ch_{i}_{j}_cut']].apply(lambda x : (x[0]*x[2]/2) + (x[1]*x[2]),axis=1)

In [90]:
test_item_weight = pd.merge(A_train1[['cust_no','item_가공식품_A_weight', 'item_신선식품_A_weight', 'item_일상용품_A_weight',
       'item_의류_A_weight']],B_train1[['cust_no','item_가공식품_B_weight', 'item_신선식품_B_weight', 'item_일상용품_B_weight',
       'item_의류_B_weight']],how='outer',on='cust_no').fillna(0)

In [91]:
test_item_weight = pd.merge(test_item_weight,C_train1[['cust_no','item_가공식품_C_weight', 'item_신선식품_C_weight', 'item_일상용품_C_weight',
       'item_의류_C_weight']],how='outer',on='cust_no').fillna(0)

In [92]:
test_item_weight

cat_dae,cust_no,item_가공식품_A_weight,item_신선식품_A_weight,item_일상용품_A_weight,item_의류_A_weight,item_가공식품_B_weight,item_신선식품_B_weight,item_일상용품_B_weight,item_의류_B_weight,item_가공식품_C_weight,item_신선식품_C_weight,item_일상용품_C_weight,item_의류_C_weight
0,1,-6.25,3.15,5.75,4.80,0.60,0.3,1.00,0.0,2.6,3.00,1.2,0.0
1,2,-6.00,-6.75,-6.75,4.75,0.00,0.0,0.45,0.0,0.0,0.00,0.0,0.0
2,3,0.00,0.00,0.00,-0.30,0.05,0.0,0.05,0.0,2.7,3.00,4.0,0.0
3,4,-4.75,-2.80,2.60,-3.40,1.00,1.0,0.70,1.8,-1.2,-3.75,-1.1,-1.4
4,5,-6.25,-4.75,-4.00,-1.65,0.00,-0.2,0.00,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,19376,0.00,0.00,0.00,0.00,0.60,0.6,0.05,0.0,0.0,0.00,0.0,0.0
19334,19377,0.05,0.00,0.00,0.05,0.05,0.0,0.00,0.0,0.0,0.00,0.0,0.0
19335,19378,0.30,0.60,0.60,0.30,0.00,0.0,0.60,0.0,0.0,0.00,0.0,0.0
19336,19379,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0


In [93]:
test_item_weight1 = pd.merge(A_test[['cust_no','item_가공식품_A_weight', 'item_신선식품_A_weight', 'item_일상용품_A_weight',
       'item_의류_A_weight']],B_test[['cust_no','item_가공식품_B_weight', 'item_신선식품_B_weight', 'item_일상용품_B_weight',
       'item_의류_B_weight']],how='outer',on='cust_no').fillna(0)

In [94]:
test_item_weight1 = pd.merge(test_item_weight1,C_test[['cust_no','item_가공식품_C_weight', 'item_신선식품_C_weight', 'item_일상용품_C_weight',
       'item_의류_C_weight']],how='outer',on='cust_no').fillna(0)

In [95]:
test_item_weight1

cat_dae,cust_no,item_가공식품_A_weight,item_신선식품_A_weight,item_일상용품_A_weight,item_의류_A_weight,item_가공식품_B_weight,item_신선식품_B_weight,item_일상용품_B_weight,item_의류_B_weight,item_가공식품_C_weight,item_신선식품_C_weight,item_일상용품_C_weight,item_의류_C_weight
0,1,3.15,-3.80,-5.25,-4.80,-1.50,1.65,-1.00,0.00,2.25,-3.0,-0.6,0.0
1,2,-4.40,-6.25,6.75,5.75,0.00,0.00,1.10,0.00,0.00,0.0,0.0,0.0
2,3,0.00,0.00,0.00,0.00,-0.05,0.00,-0.05,0.00,-4.50,-3.0,-3.2,0.0
3,4,3.40,4.00,-2.60,0.75,1.20,0.60,-0.45,-0.75,1.40,1.5,-0.9,-0.4
4,5,5.75,-0.75,2.10,1.65,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,19283,4.00,0.20,4.00,0.20,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
19352,19350,0.00,0.00,0.00,0.00,0.60,0.00,0.00,0.00,0.10,0.0,0.0,0.0
19353,19381,0.00,0.00,0.00,1.20,3.00,4.00,2.00,2.00,0.00,0.0,0.0,0.0
19354,19382,0.00,0.00,0.00,0.20,3.00,3.00,1.60,0.60,0.00,0.0,0.0,0.0


In [98]:
test_item = pd.merge(test_item_weight,test_item_weight1,how='outer',on='cust_no',suffixes=['_1','_2']).fillna(0)

In [99]:
test_item

cat_dae,cust_no,item_가공식품_A_weight_1,item_신선식품_A_weight_1,item_일상용품_A_weight_1,item_의류_A_weight_1,item_가공식품_B_weight_1,item_신선식품_B_weight_1,item_일상용품_B_weight_1,item_의류_B_weight_1,item_가공식품_C_weight_1,item_신선식품_C_weight_1,item_일상용품_C_weight_1,item_의류_C_weight_1,item_가공식품_A_weight_2,item_신선식품_A_weight_2,item_일상용품_A_weight_2,item_의류_A_weight_2,item_가공식품_B_weight_2,item_신선식품_B_weight_2,item_일상용품_B_weight_2,item_의류_B_weight_2,item_가공식품_C_weight_2,item_신선식품_C_weight_2,item_일상용품_C_weight_2,item_의류_C_weight_2
0,1,-6.25,3.15,5.75,4.80,0.60,0.3,1.00,0.0,2.6,3.00,1.2,0.0,3.15,-3.80,-5.25,-4.80,-1.50,1.65,-1.00,0.00,2.25,-3.0,-0.6,0.0
1,2,-6.00,-6.75,-6.75,4.75,0.00,0.0,0.45,0.0,0.0,0.00,0.0,0.0,-4.40,-6.25,6.75,5.75,0.00,0.00,1.10,0.00,0.00,0.0,0.0,0.0
2,3,0.00,0.00,0.00,-0.30,0.05,0.0,0.05,0.0,2.7,3.00,4.0,0.0,0.00,0.00,0.00,0.00,-0.05,0.00,-0.05,0.00,-4.50,-3.0,-3.2,0.0
3,4,-4.75,-2.80,2.60,-3.40,1.00,1.0,0.70,1.8,-1.2,-3.75,-1.1,-1.4,3.40,4.00,-2.60,0.75,1.20,0.60,-0.45,-0.75,1.40,1.5,-0.9,-0.4
4,5,-6.25,-4.75,-4.00,-1.65,0.00,-0.2,0.00,0.0,0.0,0.00,0.0,0.0,5.75,-0.75,2.10,1.65,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19367,19283,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,4.00,0.20,4.00,0.20,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0
19368,19350,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.60,0.00,0.00,0.00,0.10,0.0,0.0,0.0
19369,19381,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,1.20,3.00,4.00,2.00,2.00,0.00,0.0,0.0,0.0
19370,19382,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.20,3.00,3.00,1.60,0.60,0.00,0.0,0.0,0.0


In [100]:
for i in ['A','B','C']:
  for j in ['가공식품','신선식품',  '일상용품', '의류']:
    test_item[f'item_{j}_{i}_weight'] = (test_item[f'item_{j}_{i}_weight_1'] + test_item[f'item_{j}_{i}_weight_2'])/2

In [104]:
test_item.columns[test_item.columns.str.contains('_weight')]

Index(['item_가공식품_A_weight_1', 'item_신선식품_A_weight_1', 'item_일상용품_A_weight_1',
       'item_의류_A_weight_1', 'item_가공식품_B_weight_1', 'item_신선식품_B_weight_1',
       'item_일상용품_B_weight_1', 'item_의류_B_weight_1', 'item_가공식품_C_weight_1',
       'item_신선식품_C_weight_1', 'item_일상용품_C_weight_1', 'item_의류_C_weight_1',
       'item_가공식품_A_weight_2', 'item_신선식품_A_weight_2', 'item_일상용품_A_weight_2',
       'item_의류_A_weight_2', 'item_가공식품_B_weight_2', 'item_신선식품_B_weight_2',
       'item_일상용품_B_weight_2', 'item_의류_B_weight_2', 'item_가공식품_C_weight_2',
       'item_신선식품_C_weight_2', 'item_일상용품_C_weight_2', 'item_의류_C_weight_2',
       'item_가공식품_A_weight', 'item_신선식품_A_weight', 'item_일상용품_A_weight',
       'item_의류_A_weight', 'item_가공식품_B_weight', 'item_신선식품_B_weight',
       'item_일상용품_B_weight', 'item_의류_B_weight', 'item_가공식품_C_weight',
       'item_신선식품_C_weight', 'item_일상용품_C_weight', 'item_의류_C_weight'],
      dtype='object', name='cat_dae')

In [106]:
test_item[['cust_no','item_가공식품_A_weight', 'item_신선식품_A_weight', 'item_일상용품_A_weight',
       'item_의류_A_weight', 'item_가공식품_B_weight', 'item_신선식품_B_weight',
       'item_일상용품_B_weight', 'item_의류_B_weight', 'item_가공식품_C_weight',
       'item_신선식품_C_weight', 'item_일상용품_C_weight', 'item_의류_C_weight']].to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/test_item_weight.csv')

In [111]:
period2 = purprd[purprd['year']==2014]
period3 = purprd[purprd['period']!='2015_2']

In [112]:
period3.period.unique()

array(['2014_1', '2014_2', '2015_1'], dtype=object)

In [117]:
item_level_2 = pd.pivot_table(period2, index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').fillna(0).reset_index()
item_level_3 = pd.pivot_table(period3, index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').fillna(0).reset_index()

In [115]:
item_level_2.columns

Index(['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류',
       '의약품', '일상용품', '전문스포츠/레저', '패션잡화'],
      dtype='object', name='cat_dae')

In [118]:
train_item = item_level_2[['cust_no','가공식품', '신선식품','의류','일상용품']]
test_item = item_level_3[['cust_no','가공식품', '신선식품','의류','일상용품']]

In [123]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
for i in ['가공식품', '신선식품','의류','일상용품']:
  train_item[f'{i}_scaler'] = scaler.fit_transform(train_item[[f'{i}']])
  test_item[f'{i}_scaler'] = scaler.fit_transform(test_item[[f'{i}']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [125]:
train_item.to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/train_item_scaler.csv')
test_item.to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/test_item_scaler.csv')

In [128]:
di_item_1 = pd.pivot_table(pr14_1,index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').reset_index().fillna(0)
di_item_2 = pd.pivot_table(pr14_2,index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').reset_index().fillna(0)
di_item_3 = pd.pivot_table(pr15_1,index='cust_no',columns='cat_dae',values='pur_amount',aggfunc='sum').reset_index().fillna(0)

In [132]:
di_train = pd.merge(di_item_1,di_item_2,how='outer',on='cust_no',suffixes=['_1','_2']).fillna(0)
di_test = pd.merge(di_item_1,di_item_3,how='outer',on='cust_no',suffixes=['_1','_3']).fillna(0)

In [195]:
di_test1 = pd.merge(di_item_2,di_item_3,how='outer',on='cust_no',suffixes=['_2','_3']).fillna(0)

In [133]:
di_train

cat_dae,cust_no,가공식품_1,가구/인테리어_1,가전_1,교육/문화용품_1,기타_1,명품_1,신선식품_1,외식_1,의류_1,의약품_1,일상용품_1,전문스포츠/레저_1,패션잡화_1,가공식품_2,가구/인테리어_2,가전_2,교육/문화용품_2,기타_2,명품_2,신선식품_2,외식_2,의류_2,의약품_2,일상용품_2,전문스포츠/레저_2,패션잡화_2
0,1,3513830.0,177650.0,0.0,0.0,0.0,0.0,966425.0,288750.0,9612380.0,0.0,2209270.0,2143200.0,1013850.0,1112490.0,4770240.0,9062100.0,0.0,0.0,0.0,1265302.0,758800.0,11842680.0,8000.0,4707600.0,0.0,589100.0
1,2,1527630.0,530700.0,1421700.0,0.0,0.0,0.0,3353800.0,472600.0,3577890.0,0.0,8692420.0,3949760.0,796730.0,1088150.0,39000.0,688000.0,0.0,0.0,434150.0,2184120.0,407500.0,6203330.0,0.0,7642020.0,2704920.0,530910.0
2,3,616100.0,0.0,0.0,0.0,0.0,0.0,132165.0,0.0,80000.0,0.0,22950.0,0.0,0.0,652550.0,0.0,26900.0,0.0,0.0,0.0,253184.0,0.0,0.0,0.0,109920.0,0.0,0.0
3,4,731750.0,5200.0,0.0,0.0,0.0,0.0,635526.0,246610.0,2305650.0,20320.0,174430.0,402100.0,520520.0,432260.0,421900.0,0.0,0.0,0.0,0.0,251990.0,177560.0,1448040.0,9800.0,808210.0,613350.0,94400.0
4,5,2016000.0,0.0,49900.0,0.0,0.0,57000.0,805820.0,474240.0,542900.0,0.0,1312640.0,63080.0,138570.0,917840.0,212300.0,0.0,0.0,0.0,0.0,268440.0,240100.0,166000.0,0.0,280850.0,71200.0,69000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,19376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,149420.0,0.0,0.0,0.0,0.0,0.0,122077.0,0.0,0.0,0.0,1800.0,0.0,0.0
19334,19377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16460.0,0.0,0.0,0.0,0.0,0.0,0.0,12500.0,15900.0,0.0,172500.0,0.0,0.0
19335,19378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23800.0,0.0,0.0,0.0,0.0,0.0,67000.0,8500.0,151240.0,0.0,224220.0,0.0,0.0
19336,19379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107740.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,0.0,9500.0,600330.0,0.0,453050.0


In [134]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  di_train[f'{i}_di'] = di_train[f'{i}_2'] - di_train[f'{i}_1']
  di_test[f'{i}_di'] = di_test[f'{i}_3'] - di_test[f'{i}_1']

In [196]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  di_test1[f'{i}_di'] = di_test1[f'{i}_3'] - di_test1[f'{i}_2']

In [136]:
di_train.columns[di_train.columns.str.contains('di')]

Index(['가공식품_di', '가구/인테리어_di', '가전_di', '교육/문화용품_di', '기타_di', '명품_di',
       '신선식품_di', '외식_di', '의류_di', '의약품_di', '일상용품_di', '전문스포츠/레저_di',
       '패션잡화_di'],
      dtype='object', name='cat_dae')

In [137]:
di_train_scaler = di_train[['cust_no','가공식품_di', '가구/인테리어_di', '가전_di', '교육/문화용품_di', '기타_di', '명품_di',
       '신선식품_di', '외식_di', '의류_di', '의약품_di', '일상용품_di', '전문스포츠/레저_di',
       '패션잡화_di']]
di_test_scaler = di_test[['cust_no','가공식품_di', '가구/인테리어_di', '가전_di', '교육/문화용품_di', '기타_di', '명품_di',
       '신선식품_di', '외식_di', '의류_di', '의약품_di', '일상용품_di', '전문스포츠/레저_di',
       '패션잡화_di']]

In [197]:
di_test_scaler1 = di_test1[['cust_no','가공식품_di', '가구/인테리어_di', '가전_di', '교육/문화용품_di', '기타_di', '명품_di',
       '신선식품_di', '외식_di', '의류_di', '의약품_di', '일상용품_di', '전문스포츠/레저_di',
       '패션잡화_di']]

In [140]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  di_train_scaler[f'{i}_scaler'] = scaler.fit_transform(di_train_scaler[[f'{i}_di']])
  di_test_scaler[f'{i}_scaler'] = scaler.fit_transform(di_test_scaler[[f'{i}_di']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [198]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  di_test_scaler1[f'{i}_scaler'] = scaler.fit_transform(di_test_scaler1[[f'{i}_di']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [142]:
di_train_scaler.columns[di_train_scaler.columns.str.contains('scaler')]

Index(['가공식품_scaler', '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler',
       '기타_scaler', '명품_scaler', '신선식품_scaler', '외식_scaler', '의류_scaler',
       '의약품_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler'],
      dtype='object', name='cat_dae')

In [144]:
di_train_scaler[['cust_no','가공식품_scaler', '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler',
       '기타_scaler', '명품_scaler', '신선식품_scaler', '외식_scaler', '의류_scaler',
       '의약품_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler']].to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/amount_di_train_scaler.csv')
di_test_scaler[['cust_no','가공식품_scaler', '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler',
       '기타_scaler', '명품_scaler', '신선식품_scaler', '외식_scaler', '의류_scaler',
       '의약품_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler']].to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/amount_di_test_scaler.csv')

In [199]:
di_test_scaler1[['cust_no','가공식품_scaler', '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler',
       '기타_scaler', '명품_scaler', '신선식품_scaler', '외식_scaler', '의류_scaler',
       '의약품_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler']].to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/amount_di_test_scaler_2-3.csv')

In [157]:
di_item_1_channel = pd.pivot_table(pr14_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum').fillna(0)
di_item_2_channel = pd.pivot_table(pr14_2,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum').fillna(0)
di_item_3_channel = pd.pivot_table(pr15_1,index='cust_no',columns=['channel','cat_dae'],values='pur_amount',aggfunc='sum').fillna(0)

In [158]:
di_item_1_channel_A = di_item_1_channel['A'].reset_index()
di_item_1_channel_B = di_item_1_channel['B'].reset_index()
di_item_1_channel_C = di_item_1_channel['C'].reset_index()
di_item_2_channel_A = di_item_2_channel['A'].reset_index()
di_item_2_channel_B = di_item_2_channel['B'].reset_index()
di_item_2_channel_C = di_item_2_channel['C'].reset_index()
di_item_3_channel_A = di_item_3_channel['A'].reset_index()
di_item_3_channel_B = di_item_3_channel['B'].reset_index()
di_item_3_channel_C = di_item_3_channel['C'].reset_index()

In [160]:
train_item_channel_A = pd.merge(di_item_1_channel_A,di_item_2_channel_A,how='outer',on='cust_no',suffixes=['_1','_2']).fillna(0)
train_item_channel_B = pd.merge(di_item_1_channel_B,di_item_2_channel_B,how='outer',on='cust_no',suffixes=['_1','_2']).fillna(0)
train_item_channel_C = pd.merge(di_item_1_channel_C,di_item_2_channel_C,how='outer',on='cust_no',suffixes=['_1','_2']).fillna(0)
test_item_channel_A = pd.merge(di_item_1_channel_A,di_item_3_channel_A,how='outer',on='cust_no',suffixes=['_1','_3']).fillna(0)
test_item_channel_B = pd.merge(di_item_1_channel_B,di_item_3_channel_B,how='outer',on='cust_no',suffixes=['_1','_3']).fillna(0)
test_item_channel_C = pd.merge(di_item_1_channel_C,di_item_3_channel_C,how='outer',on='cust_no',suffixes=['_1','_3']).fillna(0)

In [213]:
test_item_channel_A_1 = pd.merge(di_item_2_channel_A,di_item_3_channel_A,how='outer',on='cust_no',suffixes=['_2','_3']).fillna(0)
test_item_channel_B_1 = pd.merge(di_item_2_channel_B,di_item_3_channel_B,how='outer',on='cust_no',suffixes=['_2','_3']).fillna(0)
test_item_channel_C_1 = pd.merge(di_item_2_channel_C,di_item_3_channel_C,how='outer',on='cust_no',suffixes=['_2','_3']).fillna(0)

In [219]:
test_item_channel_C_1

cat_dae,cust_no,가공식품_2,가구/인테리어_2,가전_2,교육/문화용품_2,신선식품_2,의류_2,의약품_2,일상용품_2,전문스포츠/레저_2,패션잡화_2,가공식품_3,가구/인테리어_3,가전_3,교육/문화용품_3,신선식품_3,의류_3,의약품_3,일상용품_3,전문스포츠/레저_3,패션잡화_3,가공식품_di,가구/인테리어_di,가전_di,교육/문화용품_di,신선식품_di,의류_di,일상용품_di,전문스포츠/레저_di,패션잡화_di,가공식품_scaler,가구/인테리어_scaler,가전_scaler,교육/문화용품_scaler,신선식품_scaler,의류_scaler,일상용품_scaler,전문스포츠/레저_scaler,패션잡화_scaler
0,1,95150.0,0.0,0.0,0.0,291342.0,0.0,0.0,13380.0,0.0,0.0,116420.0,0.0,0.0,0.0,156439.0,0.0,21800.0,12820.0,0.0,0.0,21270.0,0.0,0.0,0.0,-134903.0,0.0,-560.0,0.0,0.0,-0.000460,0.005071,0.055834,0.007497,-0.573687,0.021612,0.003952,0.002193,-0.006541
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.007106,0.005071,0.055834,0.007497,-0.032285,0.021612,0.004659,0.002193,-0.006541
2,3,650590.0,0.0,26900.0,0.0,253184.0,0.0,0.0,104920.0,0.0,0.0,374950.0,0.0,0.0,0.0,163310.0,0.0,0.0,54020.0,0.0,0.0,-275640.0,0.0,-26900.0,0.0,-89874.0,0.0,-50900.0,0.0,0.0,-0.093243,0.005071,-0.797474,0.007497,-0.392974,0.021612,-0.059565,0.002193,-0.006541
3,4,33750.0,0.0,0.0,0.0,76460.0,9900.0,0.0,5140.0,0.0,0.0,53580.0,0.0,0.0,0.0,96813.0,0.0,0.0,1900.0,0.0,0.0,19830.0,0.0,0.0,0.0,20353.0,-9900.0,-3240.0,0.0,0.0,-0.000910,0.005071,0.055834,0.007497,0.049397,-0.269720,0.000571,0.002193,-0.006541
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.007106,0.005071,0.055834,0.007497,-0.032285,0.021612,0.004659,0.002193,-0.006541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19351,19283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.007106,0.005071,0.055834,0.007497,-0.032285,0.021612,0.004659,0.002193,-0.006541
19352,19350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1080.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006769,0.005071,0.055834,0.007497,-0.032285,0.021612,0.004659,0.002193,-0.006541
19353,19381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.007106,0.005071,0.055834,0.007497,-0.032285,0.021612,0.004659,0.002193,-0.006541
19354,19382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.007106,0.005071,0.055834,0.007497,-0.032285,0.021612,0.004659,0.002193,-0.006541


In [164]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  try:
    train_item_channel_A[f'{i}_di'] = train_item_channel_A[f'{i}_2'] - train_item_channel_A[f'{i}_1']
    train_item_channel_B[f'{i}_di'] = train_item_channel_B[f'{i}_2'] - train_item_channel_B[f'{i}_1']
    train_item_channel_C[f'{i}_di'] = train_item_channel_C[f'{i}_2'] - train_item_channel_C[f'{i}_1']
    test_item_channel_A[f'{i}_di'] = test_item_channel_A[f'{i}_3'] - test_item_channel_A[f'{i}_1']
    test_item_channel_B[f'{i}_di'] = test_item_channel_B[f'{i}_3'] - test_item_channel_B[f'{i}_1']
    test_item_channel_C[f'{i}_di'] = test_item_channel_C[f'{i}_3'] - test_item_channel_C[f'{i}_1']
  except:
    pass

In [214]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  try:
    test_item_channel_A_1[f'{i}_di'] = test_item_channel_A_1[f'{i}_3'] - test_item_channel_A_1[f'{i}_2']
    test_item_channel_B_1[f'{i}_di'] = test_item_channel_B_1[f'{i}_3'] - test_item_channel_B_1[f'{i}_2']
    test_item_channel_C_1[f'{i}_di'] = test_item_channel_C_1[f'{i}_3'] - test_item_channel_C_1[f'{i}_2']
  except:
    pass

In [165]:
train_item_channel_A

cat_dae,cust_no,가공식품_1,가구/인테리어_1,가전_1,교육/문화용품_1,기타_1,명품_1,신선식품_1,외식_1,의류_1,일상용품_1,전문스포츠/레저_1,패션잡화_1,가공식품_2,가구/인테리어_2,가전_2,교육/문화용품_2,기타_2,명품_2,신선식품_2,외식_2,의류_2,일상용품_2,전문스포츠/레저_2,패션잡화_2,가공식품_di,가구/인테리어_di,가전_di,교육/문화용품_di,기타_di,명품_di,신선식품_di,외식_di,의류_di,일상용품_di,전문스포츠/레저_di,패션잡화_di
0,1,3492840.0,177650.0,0.0,0.0,0.0,0.0,787820.0,266950.0,9612380.0,2208520.0,2143200.0,1013850.0,914680.0,4770240.0,9062100.0,0.0,0.0,0.0,924860.0,702400.0,11842680.0,4534240.0,0.0,589100.0,-2578160.0,4592590.0,9062100.0,0.0,0.0,0.0,137040.0,435450.0,2230300.0,2325720.0,-2143200.0,-424750.0
1,2,1527630.0,530700.0,1421700.0,0.0,0.0,0.0,3353800.0,472600.0,3577890.0,8661920.0,3949760.0,796730.0,1088150.0,39000.0,688000.0,0.0,0.0,434150.0,2184120.0,407500.0,6203330.0,7595570.0,2704920.0,530910.0,-439480.0,-491700.0,-733700.0,0.0,0.0,434150.0,-1169680.0,-65100.0,2625440.0,-1066350.0,-1244840.0,-265820.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-80000.0,0.0,0.0,0.0
3,4,659890.0,0.0,0.0,0.0,0.0,0.0,367480.0,246610.0,2272650.0,159010.0,402100.0,520520.0,301290.0,404000.0,0.0,0.0,0.0,0.0,91530.0,177560.0,1323740.0,775050.0,613350.0,25000.0,-358600.0,404000.0,0.0,0.0,0.0,0.0,-275950.0,-69050.0,-948910.0,616040.0,211250.0,-495520.0
4,5,2016000.0,0.0,49900.0,0.0,0.0,57000.0,789820.0,474240.0,542900.0,1312640.0,63080.0,138570.0,917840.0,212300.0,0.0,0.0,0.0,0.0,268440.0,240100.0,166000.0,280850.0,71200.0,69000.0,-1098160.0,212300.0,-49900.0,0.0,0.0,-57000.0,-521380.0,-234140.0,-376900.0,-1031790.0,8120.0,-69570.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,19376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19334,19377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,12500.0,15900.0,0.0,0.0,0.0,4000.0,0.0,0.0,0.0,0.0,0.0,0.0,12500.0,15900.0,0.0,0.0,0.0
19335,19378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23800.0,0.0,0.0,0.0,0.0,0.0,67000.0,8500.0,151240.0,163220.0,0.0,0.0,23800.0,0.0,0.0,0.0,0.0,0.0,67000.0,8500.0,151240.0,163220.0,0.0,0.0
19336,19379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,449900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,0.0,0.0,0.0,449900.0


In [166]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  try:
    train_item_channel_A[f'{i}_scaler'] = scaler.fit_transform(train_item_channel_A[[f'{i}_di']])
    train_item_channel_B[f'{i}_scaler'] = scaler.fit_transform(train_item_channel_B[[f'{i}_di']])
    train_item_channel_C[f'{i}_scaler'] = scaler.fit_transform(train_item_channel_C[[f'{i}_di']])
    test_item_channel_A[f'{i}_scaler'] = scaler.fit_transform(test_item_channel_A[[f'{i}_di']])
    test_item_channel_B[f'{i}_scaler'] = scaler.fit_transform(test_item_channel_B[[f'{i}_di']])
    test_item_channel_C[f'{i}_scaler'] = scaler.fit_transform(test_item_channel_C[[f'{i}_di']])
  except:
    pass

In [215]:
for i in ['가공식품', '가구/인테리어', '가전', '교육/문화용품', '기타', '명품', '신선식품', '외식', '의류','의약품', '일상용품', '전문스포츠/레저', '패션잡화']:
  try:
    test_item_channel_A_1[f'{i}_scaler'] = scaler.fit_transform(test_item_channel_A_1[[f'{i}_di']])
    test_item_channel_B_1[f'{i}_scaler'] = scaler.fit_transform(test_item_channel_B_1[[f'{i}_di']])
    test_item_channel_C_1[f'{i}_scaler'] = scaler.fit_transform(test_item_channel_C_1[[f'{i}_di']])
  except:
    pass

In [222]:
test_item_channel_A_1.columns

Index(['cust_no', '가공식품_2', '가구/인테리어_2', '가전_2', '교육/문화용품_2', '기타_2', '명품_2',
       '신선식품_2', '외식_2', '의류_2', '일상용품_2', '전문스포츠/레저_2', '패션잡화_2', '가공식품_3',
       '가구/인테리어_3', '가전_3', '교육/문화용품_3', '기타_3', '명품_3', '신선식품_3', '외식_3',
       '의류_3', '일상용품_3', '전문스포츠/레저_3', '패션잡화_3', '가공식품_di', '가구/인테리어_di',
       '가전_di', '교육/문화용품_di', '기타_di', '명품_di', '신선식품_di', '외식_di', '의류_di',
       '일상용품_di', '전문스포츠/레저_di', '패션잡화_di', '가공식품_scaler', '가구/인테리어_scaler',
       '가전_scaler', '교육/문화용품_scaler', '기타_scaler', '명품_scaler', '신선식품_scaler',
       '외식_scaler', '의류_scaler', '일상용품_scaler', '전문스포츠/레저_scaler',
       '패션잡화_scaler'],
      dtype='object', name='cat_dae')

In [182]:
train_channel_di = pd.merge(pd.merge(train_item_channel_A[['cust_no','가공식품_scaler', '가구/인테리어_scaler',
       '가전_scaler', '교육/문화용품_scaler', '기타_scaler', '명품_scaler', '신선식품_scaler',
       '외식_scaler', '의류_scaler', '일상용품_scaler', '전문스포츠/레저_scaler',
       '패션잡화_scaler']],train_item_channel_B[['cust_no','가공식품_scaler',
       '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler', '기타_scaler',
       '명품_scaler', '신선식품_scaler', '외식_scaler', '의류_scaler', '일상용품_scaler',
       '전문스포츠/레저_scaler', '패션잡화_scaler']],how='outer',on='cust_no',suffixes=['_A','_B']),train_item_channel_C[['cust_no','가공식품_scaler',
       '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler', '신선식품_scaler',
       '의류_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler']],how='outer',on='cust_no')

In [224]:
test_channel_di_1.columns

Index(['cust_no', '가공식품_scaler_A', '가구/인테리어_scaler_A', '가전_scaler_A',
       '교육/문화용품_scaler_A', '기타_scaler_A', '명품_scaler_A', '신선식품_scaler_A',
       '외식_scaler_A', '의류_scaler_A', '일상용품_scaler_A', '전문스포츠/레저_scaler_A',
       '패션잡화_scaler_A', '가공식품_scaler_B', '가구/인테리어_scaler_B', '가전_scaler_B',
       '교육/문화용품_scaler_B', '기타_scaler_B', '명품_scaler_B', '신선식품_scaler_B',
       '외식_scaler_B', '의류_scaler_B', '일상용품_scaler_B', '전문스포츠/레저_scaler_B',
       '패션잡화_scaler_B', '가공식품_scaler', '가구/인테리어_scaler', '가전_scaler',
       '교육/문화용품_scaler', '신선식품_scaler', '의류_scaler', '일상용품_scaler',
       '전문스포츠/레저_scaler', '패션잡화_scaler'],
      dtype='object', name='cat_dae')

In [184]:
train_channel_di.columns = ['cust_no', '가공식품_scaler_A', '가구/인테리어_scaler_A', '가전_scaler_A',
       '교육/문화용품_scaler_A', '기타_scaler_A', '명품_scaler_A', '신선식품_scaler_A',
       '외식_scaler_A', '의류_scaler_A', '일상용품_scaler_A', '전문스포츠/레저_scaler_A',
       '패션잡화_scaler_A', '가공식품_scaler_B', '가구/인테리어_scaler_B', '가전_scaler_B',
       '교육/문화용품_scaler_B', '기타_scaler_B', '명품_scaler_B', '신선식품_scaler_B',
       '외식_scaler_B', '의류_scaler_B', '일상용품_scaler_B', '전문스포츠/레저_scaler_B',
       '패션잡화_scaler_B', '가공식품_scaler_C', '가구/인테리어_scaler_C', '가전_scaler_C',
       '교육/문화용품_scaler_C', '신선식품_scaler_C', '의류_scaler_C', '일상용품_scaler_C',
       '전문스포츠/레저_scaler_C', '패션잡화_scaler_C']

In [188]:
test_channel_di = pd.merge(pd.merge(test_item_channel_A[['cust_no','가공식품_scaler', '가구/인테리어_scaler', '가전_scaler',
       '교육/문화용품_scaler', '신선식품_scaler', '의류_scaler', '일상용품_scaler',
       '전문스포츠/레저_scaler', '패션잡화_scaler']],test_item_channel_B[['cust_no','가공식품_scaler', '가구/인테리어_scaler', '가전_scaler',
       '교육/문화용품_scaler', '신선식품_scaler', '의류_scaler', '일상용품_scaler',
       '전문스포츠/레저_scaler', '패션잡화_scaler']],how='outer',on='cust_no',suffixes=['_A','_B']),test_item_channel_C[['cust_no','가공식품_scaler',
       '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler', '신선식품_scaler',
       '의류_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler']],how='outer',on='cust_no')

In [190]:
test_channel_di.columns = ['cust_no', '가공식품_scaler_A', '가구/인테리어_scaler_A', '가전_scaler_A',
       '교육/문화용품_scaler_A', '신선식품_scaler_A', '의류_scaler_A', '일상용품_scaler_A',
       '전문스포츠/레저_scaler_A', '패션잡화_scaler_A', '가공식품_scaler_B',
       '가구/인테리어_scaler_B', '가전_scaler_B', '교육/문화용품_scaler_B', '신선식품_scaler_B',
       '의류_scaler_B', '일상용품_scaler_B', '전문스포츠/레저_scaler_B', '패션잡화_scaler_B',
       '가공식품_scaler_C', '가구/인테리어_scaler_C', '가전_scaler_C', '교육/문화용품_scaler_C',
       '신선식품_scaler_C', '의류_scaler_C', '일상용품_scaler_C', '전문스포츠/레저_scaler_C',
       '패션잡화_scaler_C']

In [223]:
test_channel_di_1 = pd.merge(pd.merge(test_item_channel_A_1[['cust_no','가공식품_scaler', '가구/인테리어_scaler',
       '가전_scaler', '교육/문화용품_scaler', '기타_scaler', '명품_scaler', '신선식품_scaler',
       '외식_scaler', '의류_scaler', '일상용품_scaler', '전문스포츠/레저_scaler',
       '패션잡화_scaler']],test_item_channel_B_1[['cust_no','가공식품_scaler',
       '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler', '기타_scaler',
       '명품_scaler', '신선식품_scaler', '외식_scaler', '의류_scaler', '일상용품_scaler',
       '전문스포츠/레저_scaler', '패션잡화_scaler']],how='outer',on='cust_no',suffixes=['_A','_B']),test_item_channel_C_1[['cust_no','가공식품_scaler',
       '가구/인테리어_scaler', '가전_scaler', '교육/문화용품_scaler', '신선식품_scaler',
       '의류_scaler', '일상용품_scaler', '전문스포츠/레저_scaler', '패션잡화_scaler']],how='outer',on='cust_no')

In [227]:
test_channel_di_1.columns = ['cust_no', '가공식품_scaler_A', '가구/인테리어_scaler_A', '가전_scaler_A',
       '교육/문화용품_scaler_A', '기타_scaler_A', '명품_scaler_A', '신선식품_scaler_A',
       '외식_scaler_A', '의류_scaler_A', '일상용품_scaler_A', '전문스포츠/레저_scaler_A',
       '패션잡화_scaler_A', '가공식품_scaler_B', '가구/인테리어_scaler_B', '가전_scaler_B',
       '교육/문화용품_scaler_B', '기타_scaler_B', '명품_scaler_B', '신선식품_scaler_B',
       '외식_scaler_B', '의류_scaler_B', '일상용품_scaler_B', '전문스포츠/레저_scaler_B',
       '패션잡화_scaler_B', '가공식품_scaler_C', '가구/인테리어_scaler_C', '가전_scaler_C',
       '교육/문화용품_scaler_C', '신선식품_scaler_C', '의류_scaler_C', '일상용품_scaler_C',
       '전문스포츠/레저_scaler_C', '패션잡화_scaler_C']

In [192]:
train_channel_di.to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/train_channel_di.csv')

In [193]:
test_channel_di.to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/test_channel_di.csv')

In [194]:
test_channel_di

,cust_no,가공식품_scaler_A,가구/인테리어_scaler_A,가전_scaler_A,교육/문화용품_scaler_A,신선식품_scaler_A,의류_scaler_A,일상용품_scaler_A,전문스포츠/레저_scaler_A,패션잡화_scaler_A,가공식품_scaler_B,가구/인테리어_scaler_B,가전_scaler_B,교육/문화용품_scaler_B,신선식품_scaler_B,의류_scaler_B,일상용품_scaler_B,전문스포츠/레저_scaler_B,패션잡화_scaler_B,가공식품_scaler_C,가구/인테리어_scaler_C,가전_scaler_C,교육/문화용품_scaler_C,신선식품_scaler_C,의류_scaler_C,일상용품_scaler_C,전문스포츠/레저_scaler_C,패션잡화_scaler_C
0,1,-2.463036,0.228626,0.104501,-0.010587,-0.065144,0.117522,-0.446867,-0.474093,0.061110,0.011504,0.006662,0.0897,0.023799,0.072910,-0.014778,-0.014133,0.105586,-0.009493,0.011617,0.010655,-0.039005,-0.001558,-0.117666,0.004830,0.009114,-0.005443,0.000739
1,2,-0.640089,-0.308362,0.171320,-0.010587,-1.768503,0.664361,0.278960,-0.469356,-0.121620,0.007840,0.006662,0.0897,0.023799,-0.033230,-0.014778,0.035579,0.105586,0.017792,-0.014027,0.010655,-0.039005,-0.001558,-0.054547,0.004830,0.003373,-0.005443,0.000739
2,3,-0.035815,-0.023176,-0.011034,-0.010587,-0.016622,-0.031746,-0.020036,0.010739,-0.008596,0.007840,0.006662,0.0897,0.023799,-0.033230,-0.014778,-0.014133,0.105586,-0.009493,-0.078829,0.010655,-0.039005,-0.001558,0.034141,0.004830,0.018152,-0.005443,0.000739
3,4,-0.202123,-0.023176,-0.011034,-0.010587,0.143570,-0.143708,0.039655,-0.096110,-0.171033,0.016059,0.000013,0.0897,0.023799,0.024813,0.363301,-0.004717,0.105586,-0.009493,-0.011469,0.010655,-0.039005,-0.001558,-0.469021,-0.422704,-0.000024,-0.005443,0.000739
4,5,-0.555796,-0.023176,-0.030845,-0.010587,-0.596384,-0.031812,-0.337272,-0.009079,-0.053893,0.007840,0.006662,0.0897,0.023799,-0.045776,-0.014778,-0.014133,0.105586,-0.009493,-0.014027,0.010655,-0.039005,-0.001558,-0.054547,0.004830,0.003373,-0.005443,0.000739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19365,19379,-0.026297,-0.023176,-0.011034,-0.010587,-0.016622,0.048019,-0.006453,0.010739,0.026708,0.009796,0.006662,0.0897,0.023799,-0.033230,0.188001,0.067371,0.105586,-0.009493,-0.008271,0.010655,-0.039005,-0.001558,0.037914,0.004830,0.003696,-0.005443,0.000739
19366,19380,-0.035815,-0.023176,-0.011034,-0.010587,-0.016622,-0.021285,-0.020036,0.010739,-0.008596,0.007953,0.006662,0.0897,0.023799,-0.033230,-0.014778,-0.014133,0.105586,0.080426,-0.014027,0.010655,-0.039005,-0.001558,-0.054547,0.004830,0.003373,-0.005443,0.000739
19367,19381,-0.035815,-0.023176,-0.011034,-0.010587,-0.016622,0.020729,-0.020036,0.010739,-0.008596,0.078397,0.106395,0.0897,0.035029,0.920326,0.499276,0.127051,0.183703,0.117634,-0.014027,0.010655,-0.039005,-0.001558,-0.054547,0.004830,0.003373,-0.005443,0.000739
19368,19382,-0.035815,-0.023176,-0.011034,-0.010587,-0.016622,-0.014760,-0.020036,0.036500,-0.008596,0.075922,0.012416,0.0897,0.068571,0.421180,0.084717,0.081652,0.229204,0.036707,-0.014027,0.010655,-0.039005,-0.001558,-0.054547,0.004830,0.003373,-0.005443,0.000739


In [228]:
test_channel_di_1.to_csv('/content/drive/MyDrive/lmembers/03 추가데이터_통계자료, 파생자료 등/dataset/test_channel_di_2-3.csv')

In [230]:
pd.pivot_table(purprd, index='y_month',values='pur_amount',aggfunc='sum')

,pur_amount
y_month,
2014_01,28249481923
2014_02,22817825296
2014_03,26627455524
2014_04,26412673959
2014_05,28456014251
2014_06,25677279187
2014_07,24578376885
2014_08,24884479432
2014_09,25486858340


In [232]:
pd.pivot_table(purprd, index='y_quarter',values='pur_amount',aggfunc='sum')

,pur_amount
y_quarter,
2014_1,77694762743
2014_2,80545967397
2014_3,74949714657
2014_4,96411395784
2015_1,84454475955
2015_2,84834674190
2015_3,79536551000
2015_4,98591615215


In [233]:
pd.pivot_table(purprd, index='period',values='pur_amount',aggfunc='sum')

,pur_amount
period,
2014_1,158240730140
2014_2,171361110441
2015_1,169289150145
2015_2,178128166215


In [234]:
pd.pivot_table(purprd, index='period',columns='channel',values='pur_amount',aggfunc='sum')

channel,A,B,C,D
period,,,,
2014_1,118614767910,28011051473,11505871747,109039010
2014_2,130492942290,28359883618,12305121439,203163094
2015_1,128711198230,27539338156,12762949819,275663940
2015_2,137022235600,27311413231,13487027884,307489500


In [235]:
pd.pivot_table(purprd, index='y_quarter',columns='channel',values='pur_amount',aggfunc='sum')

channel,A,B,C,D
y_quarter,,,,
2014_1,58019314630,13786999474,5842416599,46032040
2014_2,60595453280,14224051999,5663455148,63006970
2014_3,54148109530,14492140773,6226595795,82868559
2014_4,76344832760,13867742845,6078525644,120294535
2015_1,63810078350,13986804506,6522455769,135137330
2015_2,64901119880,13552533650,6240494050,140526610
2015_3,57927336480,14546798162,6916120548,146295810
2015_4,79094899120,12764615069,6570907336,161193690


In [236]:
pd.pivot_table(purprd, index='y_month',columns='channel',values='pur_amount',aggfunc='sum')

channel,A,B,C,D
y_month,,,,
2014_01,20957673260,4991336502,2287847361,12624800
2014_02,17079312000,3995862756,1727340060,15310480
2014_03,19982329370,4799800216,1827229178,18096760
2014_04,19928395250,4560673223,1903599386,20006100
2014_05,21800068960,4743682065,1891214106,21049120
2014_06,18866989070,4919696711,1868641656,21951750
2014_07,17808099440,4774052826,1969750479,26474140
2014_08,17738558600,5029211040,2089325942,27383850
2014_09,18601451490,4688876907,2167519374,29010569


In [1]:
pd.pivot_table(purprd, index='y_month',columns='durable',values='pur_amount',aggfunc='sum')

NameError: ignored